<a href="https://colab.research.google.com/github/Henil21/Natural_Language_processing/blob/main/time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
!wget 'https://raw.githubusercontent.com/Henil21/Natural_Language_processing/main/maindata.csv'

--2024-03-12 15:52:30--  https://raw.githubusercontent.com/Henil21/Natural_Language_processing/main/maindata.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 363252 (355K) [text/plain]
Saving to: ‘maindata.csv.2’

maindata.csv.2      100%[===================>] 354.74K  --.-KB/s    in 0.02s   

2024-03-12 15:52:30 (14.7 MB/s) - ‘maindata.csv.2’ saved [363252/363252]



In [67]:
csv_path = "maindata.csv"

df = pd.read_csv(csv_path)

df.head()

,name,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,windspeed,winddir,sealevelpressure,solarradiation
0,Ahmedabad,01-01-2012,26.1,11.1,19.2,26.1,11.1,19.2,6.8,45.6,0.0,17.3,342.6,1013.4,178.2
1,Ahmedabad,02-01-2012,26.1,10.5,18.8,26.1,10.5,18.8,10.0,58.7,0.0,16.2,35.8,1014.2,176.5
2,Ahmedabad,03-01-2012,27.1,11.9,19.8,26.6,11.9,19.8,11.1,60.1,0.0,14.8,55.6,1013.5,187.9
3,Ahmedabad,04-01-2012,29.1,12.5,20.8,27.5,12.5,20.6,7.9,49.0,0.0,11.2,356.2,1012.4,189.7
4,Ahmedabad,05-01-2012,29.1,14.1,21.7,27.8,14.1,21.4,11.1,54.2,0.0,11.2,13.3,1014.1,186.6


In [68]:
# df.head()
# name=list(df.columns.values)
# name

In [69]:
df=df.drop("name",axis=1)

In [70]:

df = df[[
 'date',
'tempmax',
 'tempmin',
 'temp',
 'feelslikemax',
 'feelslikemin',
 'feelslike',
 'dew',
 'humidity',
 'precip',
 'windspeed',
 'winddir',
 'sealevelpressure',
 'solarradiation']] #Require columns extracting
df['date'] = pd.to_datetime(df['date'].apply(lambda x: x.split()[0])) #Convert to datetime dtype
df.set_index('date', drop=True, inplace=True)
df.head()

<ipython-input-70-eb25adbc559d>:16: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['date'] = pd.to_datetime(df['date'].apply(lambda x: x.split()[0])) #Convert to datetime dtype


,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,windspeed,winddir,sealevelpressure,solarradiation
date,,,,,,,,,,,,,
2012-01-01,26.1,11.1,19.2,26.1,11.1,19.2,6.8,45.6,0.0,17.3,342.6,1013.4,178.2
2012-02-01,26.1,10.5,18.8,26.1,10.5,18.8,10.0,58.7,0.0,16.2,35.8,1014.2,176.5
2012-03-01,27.1,11.9,19.8,26.6,11.9,19.8,11.1,60.1,0.0,14.8,55.6,1013.5,187.9
2012-04-01,29.1,12.5,20.8,27.5,12.5,20.6,7.9,49.0,0.0,11.2,356.2,1012.4,189.7
2012-05-01,29.1,14.1,21.7,27.8,14.1,21.4,11.1,54.2,0.0,11.2,13.3,1014.1,186.6


In [72]:
from sklearn.preprocessing import MinMaxScaler

# Exclude the 'date' column if it exists and is not numerical
if 'date' in df.columns:
    columns_to_normalize = df.columns.drop('date')
else:
    columns_to_normalize = df.columns

# Initialize the MinMaxScaler
mms = MinMaxScaler()

# Fit and transform the data, excluding the 'date' column if it exists
df[columns_to_normalize] = mms.fit_transform(df[columns_to_normalize])

# Check for NaN values after normalization
print(df.isnull().sum())

# Drop rows with NaN values if any
df.dropna(inplace=True)

# Display the head of the DataFrame
print(df.head())


tempmax              0
tempmin              0
temp                 0
feelslikemax         0
feelslikemin         0
feelslike            0
dew                  0
humidity             0
precip               0
windspeed            0
winddir              0
sealevelpressure    44
solarradiation       0
dtype: int64
             tempmax   tempmin      temp  feelslikemax  feelslikemin  \
date                                                                   
2012-01-01  0.243671  0.183908  0.182186      0.151874      0.199461   
2012-02-01  0.243671  0.160920  0.165992      0.151874      0.183288   
2012-03-01  0.275316  0.214559  0.206478      0.161736      0.221024   
2012-04-01  0.338608  0.237548  0.246964      0.179487      0.237197   
2012-05-01  0.338608  0.298851  0.283401      0.185404      0.280323   

            feelslike       dew  humidity  precip  windspeed   winddir  \
date                                                                     
2012-01-01   0.134897  0.351275  0.

In [73]:
#split data into Training and testing
training_size = round(len(df) * 0.75) #75% for training
training_size
training_data = df[:training_size]
testing_data = df[training_size:]
training_data.shape, testing_data.shape

((3254, 13), (1085, 13))

In [74]:
def create_sequence(dataset, sequence_length):
    sequence = []
    labels = []

    for i in range(len(dataset) - sequence_length):
        sequence.append(dataset.iloc[i:i+sequence_length].values)  # Extracting values for the sequence
        labels.append(dataset.iloc[i+sequence_length].values)  # Extracting values for the label

    return np.array(sequence), np.array(labels)


In [75]:
sequence_length = 50  # Set your desired sequence length
train_seq, train_label = create_sequence(training_data, sequence_length)
test_seq, test_label = create_sequence(testing_data, sequence_length)


In [76]:
import tensorflow as tf

In [77]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(train_seq.shape[1], train_seq.shape[2])))
model.add(Dropout(0.1))
model.add(LSTM(units=50))
model.add(Dense(train_label.shape[1]))  # Output dimension changed to match label dimension

model.compile(loss='mean_squared_error',
                        optimizer=tf.keras.optimizers.Adam(0.00001), metrics=['mean_absolute_error'])

model.summary()
# model.fit(train_seq, train_label, epochs=100, validation_data=(test_seq, test_label), verbose=1)


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_20 (LSTM)              (None, 50, 50)            12800     
                                                                 
 dropout_10 (Dropout)        (None, 50, 50)            0         
                                                                 
 lstm_21 (LSTM)              (None, 50)                20200     
                                                                 
 dense_10 (Dense)            (None, 13)                663       
                                                                 
Total params: 33663 (131.50 KB)
Trainable params: 33663 (131.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [79]:
model.fit(train_seq, train_label, epochs = 20, validation_data = (test_seq, test_label), verbose = 1)

Epoch 1/20
101/101 [==============================] - 1s 10ms/step - loss: 0.0135 - mean_absolute_error: 0.0804 - val_loss: 0.0130 - val_mean_absolute_error: 0.0783
Epoch 2/20
101/101 [==============================] - 1s 9ms/step - loss: 0.0133 - mean_absolute_error: 0.0797 - val_loss: 0.0129 - val_mean_absolute_error: 0.0779
Epoch 3/20
101/101 [==============================] - 1s 9ms/step - loss: 0.0131 - mean_absolute_error: 0.0792 - val_loss: 0.0127 - val_mean_absolute_error: 0.0772
Epoch 4/20
101/101 [==============================] - 1s 11ms/step - loss: 0.0130 - mean_absolute_error: 0.0787 - val_loss: 0.0126 - val_mean_absolute_error: 0.0767
Epoch 5/20
101/101 [==============================] - 1s 12ms/step - loss: 0.0128 - mean_absolute_error: 0.0779 - val_loss: 0.0124 - val_mean_absolute_error: 0.0760
Epoch 6/20
101/101 [==============================] - 1s 9ms/step - loss: 0.0127 - mean_absolute_error: 0.0775 - val_loss: 0.0123 - val_mean_absolute_error: 0.0755
Epoch 7/20
10